## Лабораторная работа №2 (Проведение исследований с логистической и линейной регрессией)



## Классификация

# 2. Создание бейзлайна и оценка качества

**a. Обучение модели sklearn**

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Загружаем датасет
df = pd.read_csv('credit_risk_dataset.csv')

# Кодируем категориальные признаки
for col in ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# Заполняем пропуски медианой по числовым колонкам
df = df.fillna(df.median(numeric_only=True))

# Разделяем на признаки (X) и целевой класс (y)
X = df.drop('loan_status', axis=1)
y = df['loan_status']

# Делим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Масштабируем признаки
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Обучаем
log_reg = LogisticRegression(
    max_iter=1000,
    C=1.0,
    n_jobs=-1,
    solver='lbfgs'
)
log_reg.fit(X_train, y_train)

# Делаем предсказания классов и вероятностей
y_pred = log_reg.predict(X_test)
y_pred_proba = log_reg.predict_proba(X_test)[:, 1]

# метрики
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("Логистическая регрессия (бейзлайн)")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

Логистическая регрессия (бейзлайн)
Accuracy: 0.8449
Precision: 0.7241
Recall: 0.4669
F1-Score: 0.5678
ROC-AUC: 0.8516


**b. Оценка качества бейзлайна**

- Accuracy: 0.8449
- Precision: 0.7241
- Recall: 0.4669
- F1-Score: 0.5678
- ROC-AUC: 0.8516

Логистичсекая регрессия с бейзлайном показывает нормальный результат, но recall уходящий за половину это не есть хорошо.


# 3. Улучшение бейзлайна

**a. Формулировка гипотез**

**Гипотеза 1: Препроцессинг**

Можно сделать более аккуратную подготовку данных: убрать явные выбросы по возрасту и стажу (например, person_age < 100, person_emp_length < 100), заполнить пропуски только в числовых колонках и заменить Label Encoding на One-hot encoding для категориальных признаков.

**Гипотеза 2: Подбор гиперпараметров**

Подобрать коэффициент регуляризации с помощью кросс-валидации по F1-Score. Проверить несколько значений C (например, 0.01, 0.1, 1, 10) и выбрать то, которое даёт лучший баланс Precision/Recall на обучающей выборке, более подходящая регуляризация поможет модели не переобучаться и чутка поднять качество на тесте.

**Гипотеза 3: Учёт дисбаланса классов**

Попробовать вариант с учётом дисбаланса через параметр class_weight='balanced'. Идея в том, что модель начнёт сильнее учитывать ошибки по дефолтным клиентам, которых меньше в выборки и таким образом  поднять Recall, пусть даже за счёт небольшого падения Precision и Accuracy.

**Гипотеза 4: Сдвиг порога классификации**

Вместо фиксированного порога 0.5 для вероятности дефолта попробовать подобрать порог по валидации (например, перебрать значения от 0.3 до 0.7 и посмотреть F1-Score и Recall). При более низком пороге модель начнёт ловить больше проблемных клиентов, пусть даже увеличится количество ложных срабатываний.

**b. Проверка гипотез**

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Загружаем датасет
df = pd.read_csv('credit_risk_dataset.csv')

# Смотрим, сколько записей до удаления выбросов
print("До удаления выбросов:", len(df))

# Удаляем явно странные значения по возрасту и стажу
df = df[(df['person_age'] < 100) & (df['person_emp_length'] < 100)]

print("После удаления выбросов:", len(df))

# Заполняем пропуски только в числовых колонках
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# One-hot encoding для категориальных признаков
df = pd.get_dummies(
    df,
    columns=['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file'],
    drop_first=False
)

# Разделяем на признаки и целевой класс
X = df.drop('loan_status', axis=1)
y = df['loan_status']

print(f"Количество признаков после One-hot: {X.shape[1]}")

# Делим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Масштабируем признаки
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Обучаем логистическую регрессию на данных после препроцессинга
log_reg_prep = LogisticRegression(
    max_iter=1000,
    C=1.0,
    n_jobs=-1,
    solver='lbfgs'
)
log_reg_prep.fit(X_train, y_train)

# Предсказания и вероятности
y_pred = log_reg_prep.predict(X_test)
y_pred_proba = log_reg_prep.predict_proba(X_test)[:, 1]

# метрики
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("Гипотеза 1: препроцессинг")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

До удаления выбросов: 32581
После удаления выбросов: 31679
Количество признаков после One-hot: 26
Гипотеза 1: препроцессинг
Accuracy: 0.8701
Precision: 0.7748
Recall: 0.5597
F1-Score: 0.6499
ROC-AUC: 0.8745


**Вывод по гипотезе 1:**

После удаления выбросов и перехода на One-hot encoding качество лог регрессии заметно подросло по всем основным метрикам по сравнению с базовым бейзлайном.

Гипотезу можно считать удачной, берём.


In [3]:
# Гипотеза 2: подбор гиперпараметров

from sklearn.model_selection import cross_val_score

# Набор значений C для проверки
C_values = [0.01, 0.1, 1.0, 3.0, 10.0]
results = []

print("Проверка разных значений C (5-fold CV, F1-Score):\n")

for C in C_values:
    log_reg_cv = LogisticRegression(
        max_iter=1000,
        C=C,
        n_jobs=-1,
        solver='lbfgs'
    )
    # Кросс-валидация по F1-Score
    cv_scores = cross_val_score(
        log_reg_cv,
        X_train, y_train,
        cv=5,
        scoring='f1'
    )
    mean_score = cv_scores.mean()
    results.append({'C': C, 'mean_f1': mean_score})
    print(f"C={C:4}: F1-Score (5-fold CV) = {mean_score:.4f}")

# Находим лучшее C по среднему F1
best = max(results, key=lambda x: x['mean_f1'])
best_C = best['C']
best_f1_cv = best['mean_f1']

print(f"\nЛучший C = {best_C} (F1-Score на кросс-валидации = {best_f1_cv:.4f})")

# Обучаем финальную модель с лучшим C
log_reg_best = LogisticRegression(
    max_iter=1000,
    C=best_C,
    n_jobs=-1,
    solver='lbfgs'
)
log_reg_best.fit(X_train, y_train)

# Предсказания и вероятности на тесте
y_pred = log_reg_best.predict(X_test)
y_pred_proba = log_reg_best.predict_proba(X_test)[:, 1]

# Метрики на тестовой выборке
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f"\nГипотеза 2: подбор C (C = {best_C})")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

Проверка разных значений C (5-fold CV, F1-Score):

C=0.01: F1-Score (5-fold CV) = 0.6320
C= 0.1: F1-Score (5-fold CV) = 0.6397
C= 1.0: F1-Score (5-fold CV) = 0.6394
C= 3.0: F1-Score (5-fold CV) = 0.6394
C=10.0: F1-Score (5-fold CV) = 0.6394

Лучший C = 0.1 (F1-Score на кросс-валидации = 0.6397)

Гипотеза 2: подбор C (C = 0.1)
Accuracy: 0.8699
Precision: 0.7757
Recall: 0.5575
F1-Score: 0.6488
ROC-AUC: 0.8745


**Вывод по гипотезе 2:**

Подбор коэффицента через кросс-валидацию дал совсем небольшой, но всё-таки прирост по F1-Score. Фактически эта гипотеза не даёт какого‑то ощутимого усиления модели, но я понял, что лучше взять значение коэффицента 0.1

Гипотезу 2 берём.

In [4]:
# Гипотеза 3: учет дисбаланса классов через class_weight='balanced'

from sklearn.linear_model import LogisticRegression

# Используем те же X_train, X_test, y_train, y_test после препроцессинга (гипотеза 1)
# и тот же лучший C из гипотезы 2 (best_C)

print(f"Используем C = {best_C} и class_weight='balanced'")

log_reg_balanced = LogisticRegression(
    max_iter=1000,
    C=best_C,
    n_jobs=-1,
    solver='lbfgs',
    class_weight='balanced'
)
log_reg_balanced.fit(X_train, y_train)

# Предсказания и вероятности
y_pred = log_reg_balanced.predict(X_test)
y_pred_proba = log_reg_balanced.predict_proba(X_test)[:, 1]

# Метрики
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("Гипотеза 3: class_weight='balanced'")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

Используем C = 0.1 и class_weight='balanced'
Гипотеза 3: class_weight='balanced'
Accuracy: 0.8123
Precision: 0.5445
Recall: 0.7890
F1-Score: 0.6443
ROC-AUC: 0.8763


**Вывод по гипотезе 3**

После включения class_weight='balanced' логистическая регрессия начала гораздо сильнее уделять внимание редкому классу, это видно по Recall, его показатели заметно выросли, но просадку по Accuracy и Precision я простить не могу, потому модель по сути помечает нормальных ребят как рискованных.

Гипотеза 3 не подтверждена.

In [5]:
# Гипотеза 4: сдвиг порога классификации

import numpy as np

# Считаем вероятности дефолта для тестовой выборки
y_proba_base = log_reg_best.predict_proba(X_test)[:, 1]

# Набор порогов, которые будем проверять
thresholds = np.arange(0.30, 0.71, 0.05)

results_thr = []

print("Проверка разных порогов для вероятности дефолта:\n")

for thr in thresholds:
    # Переводим вероятности в классы по заданному порогу
    y_pred_thr = (y_proba_base >= thr).astype(int)

    acc = accuracy_score(y_test, y_pred_thr)
    prec = precision_score(y_test, y_pred_thr)
    rec = recall_score(y_test, y_pred_thr)
    f1_thr = f1_score(y_test, y_pred_thr)

    results_thr.append({
        'thr': thr,
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'f1': f1_thr
    })

    print(f"Порог={thr:.2f} | Accuracy={acc:.4f} | Precision={prec:.4f} | Recall={rec:.4f} | F1={f1_thr:.4f}")

# Ищем порог с максимальным F1-Score
best_thr_obj = max(results_thr, key=lambda x: x['f1'])
best_thr = best_thr_obj['thr']

print(f"\nЛучший порог по F1-Score: {best_thr:.2f}")

# Считаем метрики для лучшего порога ещё раз, аккуратно выводим
y_pred_best_thr = (y_proba_base >= best_thr).astype(int)

acc_best = accuracy_score(y_test, y_pred_best_thr)
prec_best = precision_score(y_test, y_pred_best_thr)
rec_best = recall_score(y_test, y_pred_best_thr)
f1_best = f1_score(y_test, y_pred_best_thr)
roc_auc_best = roc_auc_score(y_test, y_proba_base)  # ROC-AUC не зависит от порога

print(f"\nГипотеза 4: сдвиг порога (threshold = {best_thr:.2f})")
print(f"Accuracy: {acc_best:.4f}")
print(f"Precision: {prec_best:.4f}")
print(f"Recall: {rec_best:.4f}")
print(f"F1-Score: {f1_best:.4f}")
print(f"ROC-AUC: {roc_auc_best:.4f}")

Проверка разных порогов для вероятности дефолта:

Порог=0.30 | Accuracy=0.8521 | Precision=0.6360 | Recall=0.7333 | F1=0.6812
Порог=0.35 | Accuracy=0.8632 | Precision=0.6774 | Recall=0.6967 | F1=0.6869
Порог=0.40 | Accuracy=0.8695 | Precision=0.7162 | Recall=0.6527 | F1=0.6830
Порог=0.45 | Accuracy=0.8706 | Precision=0.7444 | Recall=0.6081 | F1=0.6694
Порог=0.50 | Accuracy=0.8699 | Precision=0.7757 | Recall=0.5575 | F1=0.6488
Порог=0.55 | Accuracy=0.8687 | Precision=0.8039 | Recall=0.5165 | F1=0.6289
Порог=0.60 | Accuracy=0.8630 | Precision=0.8248 | Recall=0.4623 | F1=0.5925
Порог=0.65 | Accuracy=0.8586 | Precision=0.8591 | Recall=0.4110 | F1=0.5560
Порог=0.70 | Accuracy=0.8504 | Precision=0.8812 | Recall=0.3531 | F1=0.5042

Лучший порог по F1-Score: 0.35

Гипотеза 4: сдвиг порога (threshold = 0.35)
Accuracy: 0.8632
Precision: 0.6774
Recall: 0.6967
F1-Score: 0.6869
ROC-AUC: 0.8745


**Вывод по гипотезе 4**

Перебор порогов показал, что оптимальное значение 0.35, есть баланс между метриками.

Гипотезу 4 берём.


**c., d и e. Формирование, обучение и оценка качества улучшенного бейзлайна**

In [6]:
# Улучшенный бейзлайн логистической регрессии
# Гипотеза 1 (препроцессинг + One-hot) + гипотеза 2 (C=0.1) + гипотеза 4 (порог 0.35)

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Загружаем датасет
df = pd.read_csv('credit_risk_dataset.csv')

# Удаляем выбросы по возрасту и стажу
df = df[(df['person_age'] < 100) & (df['person_emp_length'] < 100)]

# Заполняем пропуски в числовых колонках
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# One-hot encoding для категориальных признаков
df = pd.get_dummies(
    df,
    columns=['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file'],
    drop_first=False
)

# Разделяем на признаки и целевой класс
X = df.drop('loan_status', axis=1)
y = df['loan_status']

# Делим на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Масштабируем признаки
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Обучаем логистическую регрессию с подобранным C
log_reg_final = LogisticRegression(
    max_iter=1000,
    C=0.1,
    n_jobs=-1,
    solver='lbfgs'
)
log_reg_final.fit(X_train, y_train)

# Получаем вероятности дефолта
y_proba = log_reg_final.predict_proba(X_test)[:, 1]

# Используем порог из гипотезы 4
threshold = 0.35
y_pred = (y_proba >= threshold).astype(int)

# Метрики
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)

print("Улучшенный бейзлайн")
print(f"Порог: {threshold}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

Улучшенный бейзлайн
Порог: 0.35
Accuracy: 0.8632
Precision: 0.6774
Recall: 0.6967
F1-Score: 0.6869
ROC-AUC: 0.8745


**f. Сравнение результатов с пунктом 2:**

Базовый бейзлайн:

- Accuracy: 0.8449  
- Precision: 0.7241  
- Recall: 0.4669  
- F1-Score: 0.5678  
- ROC-AUC: 0.8516  

Улучшенный бейзлайн:

- Accuracy: 0.8632  
- Precision: 0.6774  
- Recall: 0.6967  
- F1-Score: 0.6869  
- ROC-AUC: 0.8745  

По сравнению с базовым бейзлайном Recall вырос примерно с 0.47 до 0.70, а F1-Score с 0.57 до 0.69, то есть модель стала намного лучше ловить дефолты и в целом точнее работать с проблемным классом. Accuracy тоже немного подрос (с 0.84 до 0.86), ROC-AUC улучшился, а вот Precision ожидаемо чуть просел, так как модель стала чаще относить клиентов к дефолтным.

**g. Выводы**

Логистическая регрессия в базовом виде дала более‑менее рабочий результат, но сильно проседала по Recall: почти половина дефолтов оставалась незамеченной. После нормального препроцессинга (удаление выбросов, One-hot для категориальных признаков), подбора коэффицента и настройки порога качества стало заметно лучше: выросли Recall и F1-Score, немного подтянулись Accuracy и ROC-AUC.

# 4. Имплементация алгоритма машинного обучения

**a. Имплементация логистической регрессии**

In [7]:
import numpy as np

class LogisticRegressionCustom:

    def __init__(self, lr=0.01, n_iter=2000):
        """
        lr: скорость обучения
        n_iter: количество итераций градиентного спуска
        """
        self.lr = lr
        self.n_iter = n_iter
        self.w = None  # веса
        self.b = None  # свободный член

    def sigmoid(self, z):
        """Сигмоида для перевода линейной комбинации признаков в вероятность."""
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        """
        Обучение модели на тренировочных данных.
        X: матрица признаков (num_samples x num_features)
        y: вектор меток (0/1)
        """
        n_samples, n_features = X.shape

        # Инициализируем веса нулями
        self.w = np.zeros(n_features)
        self.b = 0.0

        # Градиентный спуск
        for _ in range(self.n_iter):
            # Линейная комбинация признаков
            linear = np.dot(X, self.w) + self.b
            # Преобразуем в вероятности
            y_pred = self.sigmoid(linear)

            # Градиенты по w и b (производные логистической потерь)
            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / n_samples) * np.sum(y_pred - y)

            # Обновляем параметры
            self.w -= self.lr * dw
            self.b -= self.lr * db

    def predict_proba(self, X):
        """Возвращает вероятность класса 1 для каждого объекта."""
        linear = np.dot(X, self.w) + self.b
        return self.sigmoid(linear)

    def predict(self, X, threshold=0.5):
        """
        Предсказываем классы (0/1) по заданному порогу.
        По умолчанию порог 0.5.
        """
        proba = self.predict_proba(X)
        return (proba >= threshold).astype(int)

**b. Обучение имплементированной модели с базовым бейзлайном**

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Загружаем датасет
df_base = pd.read_csv('credit_risk_dataset.csv')

# Кодируем категориальные признаки (как в базовом бейзлайне из пункта 2)
for col in ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']:
    le = LabelEncoder()
    df_base[col] = le.fit_transform(df_base[col])

# Заполняем пропуски медианой по числовым колонкам
df_base = df_base.fillna(df_base.median(numeric_only=True))

# Разделяем на признаки и целевой класс
X_base = df_base.drop('loan_status', axis=1)
y_base = df_base['loan_status']

# Делим на train/test
X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(
    X_base, y_base,
    test_size=0.2,
    random_state=42,
    stratify=y_base
)

# Масштабируем признаки
scaler_base = StandardScaler()
X_train_base = scaler_base.fit_transform(X_train_base)
X_test_base = scaler_base.transform(X_test_base)

# Переводим в numpy
X_train_base = np.array(X_train_base)
X_test_base = np.array(X_test_base)
y_train_base = np.array(y_train_base)
y_test_base = np.array(y_test_base)

# Обучаем имплементированную логистическую регрессию
log_reg_custom_base = LogisticRegressionCustom(lr=0.01, n_iter=2000)
log_reg_custom_base.fit(X_train_base, y_train_base)

print("Имплементированная логистическая регрессия на базовом бейзлайне обучена")

Имплементированная логистическая регрессия на базовом бейзлайне обучена


**c. Оценка качества имплементированной модели с базовым бейзлайном**

In [9]:
# Предсказания на тестовой выборке с порогом 0.5
y_pred_base = log_reg_custom_base.predict(X_test_base, threshold=0.5)
y_proba_base = log_reg_custom_base.predict_proba(X_test_base)

print("Логистическая регрессия (базовый бейзлайн)")
print(f"Accuracy: {accuracy_score(y_test_base, y_pred_base):.4f}")
print(f"Precision: {precision_score(y_test_base, y_pred_base):.4f}")
print(f"Recall: {recall_score(y_test_base, y_pred_base):.4f}")
print(f"F1-Score: {f1_score(y_test_base, y_pred_base):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test_base, y_proba_base):.4f}")

Логистическая регрессия (базовый бейзлайн)
Accuracy: 0.8438
Precision: 0.7306
Recall: 0.4501
F1-Score: 0.5570
ROC-AUC: 0.8467


**d. Сравнение результатов с пунктом 2**

В пункте 2 библиотечная логистическая регрессия давала такие метрики: Accuracy = 0.8449, Precision = 0.7241, Recall = 0.4669, F1-Score = 0.5678, ROC-AUC = 0.8516.​​ У моей логистической регрессии на том же базовом бейзлайне получились очень близкие значения: Accuracy = 0.8438, Precision = 0.7306, Recall = 0.4501, F1-Score = 0.5570, ROC-AUC = 0.8467.

Разница по всем метрикам укладывается в несколько сотых: где‑то чуть выше Precision, где‑то чуть ниже Recall и ROC-AUC, но общая картина совпадает, так что результаты ожидаемы.

**e. Выводы**

Реализованная логистическая регрессия на базовом ведёт себя так же, как и библиотечная версия из sklearn: метрики отличаются совсем немного и показывают тот же уровень качества.​

**f. и g. Добавление техник из улучшенного бейзлайна и обучение имплементированной модели**

In [ ]:
# Добавление техник из улучшенного бейзлайна и обучение

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Загружаем датасет
df = pd.read_csv('credit_risk_dataset.csv')

# Удаляем выбросы по возрасту и стажу
df = df[(df['person_age'] < 100) & (df['person_emp_length'] < 100)]

# Заполняем пропуски только в числовых колонках
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# One-hot encoding для категориальных признаков
df = pd.get_dummies(
    df,
    columns=['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file'],
    drop_first=False
)

# Разделяем на признаки и целевой класс
X = df.drop('loan_status', axis=1)
y = df['loan_status']

# Делим на train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Масштабируем признаки
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Переводим в numpy
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Обучаем логистическую регрессию на улучшенном бейзлайне
log_reg_custom_improved = LogisticRegressionCustom(lr=0.01, n_iter=2000)
log_reg_custom_improved.fit(X_train, y_train)

print("Логистическая регрессия (улучшенный бейзлайн) обучена")
print("Размер тестового набора:", len(X_test))

Логистическая регрессия (улучшенный бейзлайн) обучена
Размер тестового набора: 6336


**h. Оценка качества имплементированной модели с улучшенным бейзлайном**

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Порог взяли из гипотезы 4
threshold = 0.35

# Вероятности и предсказания
y_proba_improved = log_reg_custom_improved.predict_proba(X_test)
y_pred_improved = (y_proba_improved >= threshold).astype(int)

print("Логистическая регрессия (улучшенный бейзлайн)")
print(f"Порог: {threshold}")
print(f"Accuracy: {accuracy_score(y_test, y_pred_improved):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_improved):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_improved):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_improved):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_proba_improved):.4f}")

Логистическая регрессия (улучшенный бейзлайн)
Порог: 0.35
Accuracy: 0.8562
Precision: 0.6523
Recall: 0.7121
F1-Score: 0.6809
ROC-AUC: 0.8710


**i. Сравнение результатов c пунктом 3**

В пункте 3 для улучшенного бейзлайна логистической регрессии из sklearn метрики были такими: Accuracy = 0.8632, Precision = 0.6774, Recall = 0.6967, F1-Score = 0.6869, ROC-AUC = 0.8745.​​ У реализованной мною логистической регрессии получилось: Accuracy = 0.8562, Precision = 0.6523, Recall = 0.7121, F1-Score = 0.6809, ROC-AUC = 0.8710.

Получается, что собственная реализация показала примерно те жже цифры: Recall даже немного выше (0.7121 против 0.6967), а Accuracy, Precision, F1 и ROC-AUC чуть ниже, но в пределах пары сотых.

**j. Выводы**

Логистическая регрессия, реализованная вручную, на улучшенном бейзлайне показывает качество, сопоставимое с библиотечной моделью: и по базовому варианту, и после добавления препроцессинга и настройки порога метрики остаются на том же уровне, что подтверждает корректную имплементацию.

## Регрессия

# 2. Создание бейзлайна и оценка качества

**a. Обучение модели sklearn**

In [10]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Загружаем датасет
df = pd.read_csv('Job_Market_India.csv')

# Уберём лишние столбцы, которые сейчас использовать не будем
# (дата записи и тренд зарплаты по времени)
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)

# Категориальные признаки в датасете
cat_cols = ['Company_Name', 'Job_Role', 'Experience_Level', 'City']
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# Заполняем возможные пропуски медианой по числовым колонкам
df = df.fillna(df.median(numeric_only=True))

# Признаки и целевая переменная
X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']

# Разделение на train / test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

# Масштабирование признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Базовая модель: линейная регрессия
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

# Предсказания на тестовой выборке
y_pred = linreg.predict(X_test_scaled)

# Метрики качества
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print("Линейная регрессия (бейзлайн)")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")
print(f"MAPE: {mape:.2f}")

Линейная регрессия (бейзлайн)
MAE:  593488.92
RMSE: 791551.53
R²:   0.0047
MAPE: 60.97


**b. Оценка качества модели**

Линейная регрессия в базовой конфигурации предсказывает зарплаты довольно грубо: средняя абсолютная ошибка почти 600 тысяч рупий, RMSE ещё выше (около 790 тысяч), а MAPE около 61%, то есть модель в среднем промахивается по зарплате примерно на 60% от реального значения.  При этом R² ≈ 0.0047 практически равен нулю, что говорит о том, что текущий набор признаков и простой Label Encoding почти не объясняют разброс зарплат в датасете. Надо улучшать, идём дальше.

# 3. Улучшение бейзлайна

**a. Формулирование гипотез**

**Гипотеза 1: Препроцессинг**

Удаление выбросов по Salary_INR (например, оставив значения между 1% и 99% перцентилями) и замена LabelEncoder на OneHotEncoder улучшат качество, так как линейная модель чувствительна к выбросам и неверной интерпретации категорий как чисел.

**Гипотеза 2: Новые признаки**

Добавление осмысленных признаков (avg_salary_by_city, avg_salary_by_role, demand_remote_ratio), которые уже показали себя в первой лабе, поможет модели уловить скрытые зависимости и улучшит предсказания, так как эти признаки несут в себе больше контекста, чем исходные.

**Гипотеза 3: Логарифмирование цели**

Логарифмирование Salary_INR сделает распределение целевой переменной более похожим на нормальное, что стабилизирует линейную модель, уменьшит влияние экстремально высоких зарплат и улучшит метрики, особенно MAPE и R².​​

**Гипотеза 4: Полиномиальные признаки:**

Возможно, зависимость зарплаты от некоторых числовых признаков (например, Demand_Index) нелинейная. Если добавить полиномиальные признаки (например, квадраты исходных числовых фичей), то линейная модель сможет уловить эту нелинейность, что приведёт к росту R².

**b. Проверка гипотез**

In [11]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Загружаем заново
df = pd.read_csv('Job_Market_India.csv')

# Дропаем лишнее
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)

# 1. Удаление выбросов (оставляем с 1-го по 99-й перцентиль)
print(f"Размер датасета до удаления выбросов: {len(df)}")
q1 = df['Salary_INR'].quantile(0.01)
q99 = df['Salary_INR'].quantile(0.99)
df = df[(df['Salary_INR'] >= q1) & (df['Salary_INR'] <= q99)]
print(f"Размер датасета после удаления выбросов: {len(df)}")

# Заполняем пропуски
df = df.fillna(df.median(numeric_only=True))

# 2. One-hot encoding
cat_cols = ['Company_Name', 'Job_Role', 'Experience_Level', 'City']
df = pd.get_dummies(df, columns=cat_cols, drop_first=False)

# Признаки и цель
X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']

print(f"Количество признаков после One-hot: {X.shape[1]}")

# Train / Test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Линейная регрессия
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

y_pred = linreg.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print("Линейная регрессия (Гипотеза 1: выбросы + One-hot)")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")
print(f"MAPE: {mape:.2f}")

Размер датасета до удаления выбросов: 30000
Размер датасета после удаления выбросов: 29400
Количество признаков после One-hot: 66
Линейная регрессия (Гипотеза 1: выбросы + One-hot)
MAE:  391436.04
RMSE: 489428.13
R²:   0.5437
MAPE: 39.19


**Вывод по гипотезе 1:**

После того как удалили 1% самых маленьких и самых больших зарплат и заменили LabelEncoder на OneHotEncoder, метрики заметно улучшились.

Гипотезу 1 используем.

In [12]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Загружаем датасет
df = pd.read_csv('Job_Market_India.csv')
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)

# 1. Удаление выбросов (1-99 перцентиль)
q1 = df['Salary_INR'].quantile(0.01)
q99 = df['Salary_INR'].quantile(0.99)
df = df[(df['Salary_INR'] >= q1) & (df['Salary_INR'] <= q99)]

# Заполнение пропусков медианой
df = df.fillna(df.median(numeric_only=True))

# 2. Новые признаки
# Средняя зарплата по городу и по роли
df['avg_salary_by_city'] = df.groupby('City')['Salary_INR'].transform('mean')
df['avg_salary_by_role'] = df.groupby('Job_Role')['Salary_INR'].transform('mean')

# Комбинация спроса и удалёнки
df['demand_remote_ratio'] = df['Demand_Index'] * (df['Remote_Option_Flag'] + 1)

print("Примеры новых признаков:")
print(f"- avg_salary_by_city (min/max): {df['avg_salary_by_city'].min():.0f} / {df['avg_salary_by_city'].max():.0f}")
print(f"- demand_remote_ratio (min/max): {df['demand_remote_ratio'].min()} / {df['demand_remote_ratio'].max()}")

# 3. One-hot encoding
df = pd.get_dummies(df, columns=['Company_Name', 'Job_Role', 'Experience_Level', 'City'], drop_first=False)

# Разделение X и y
X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']

print(f"Признаков после One-hot и добавления новых: {X.shape[1]}")

# Train / Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Обучение линейной регрессии
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

# Предсказание и метрики
y_pred = linreg.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print("Линейная регрессия (Гипотеза 2: новые признаки)")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")
print(f"MAPE: {mape:.2f}")

Примеры новых признаков:
- avg_salary_by_city (min/max): 1255915 / 1296444
- demand_remote_ratio (min/max): 10 / 198
Признаков после One-hot и добавления новых: 69
Линейная регрессия (Гипотеза 2: новые признаки)
MAE:  391475.43
RMSE: 489404.83
R²:   0.5438
MAPE: 39.20


**Вывод по гипотезе 2:**

Добавление новых признаков практически ничего не изменило. Метрики остались почти такими же, как в первой гипотезе: MAE около 391 тысячи, R² замер на уровне 0.5438 (был 0.5437).​ Видимо, для линейной регрессии эти признаки оказались избыточными.

Гипотеза 2 неудачная.

In [13]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

df = pd.read_csv('Job_Market_India.csv')
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)

# 1. Удаление выбросов
q1 = df['Salary_INR'].quantile(0.01)
q99 = df['Salary_INR'].quantile(0.99)
df = df[(df['Salary_INR'] >= q1) & (df['Salary_INR'] <= q99)]
df = df.fillna(df.median(numeric_only=True))

# 2. Логарифмирование цели (log1p)
df['Salary_INR_log'] = np.log1p(df['Salary_INR'])

print(f"Зарплата (min/max): {df['Salary_INR'].min():.0f} / {df['Salary_INR'].max():.0f}")
print(f"Log зарплаты (min/max): {df['Salary_INR_log'].min():.2f} / {df['Salary_INR_log'].max():.2f}")

# 3. One-hot encoding
df = pd.get_dummies(df, columns=['Company_Name', 'Job_Role', 'Experience_Level', 'City'], drop_first=False)

# Убираем исходную зарплату из признаков, предсказываем логарифм
X = df.drop(['Salary_INR', 'Salary_INR_log'], axis=1)
y_log = df['Salary_INR_log']
y_original = df['Salary_INR']

# Сплит
X_train, X_test, y_train_log, y_test_log, y_train_orig, y_test_orig = train_test_split(
    X, y_log, y_original, test_size=0.2, random_state=42
)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Линейная регрессия на логарифме
linreg_log = LinearRegression()
linreg_log.fit(X_train_scaled, y_train_log)

# Предсказание логарифма
y_pred_log = linreg_log.predict(X_test_scaled)

# Возвращаем предсказания в обычную шкалу
y_pred = np.expm1(y_pred_log)

# Метрики считаем на исходных (реальных) деньгах
mae = mean_absolute_error(y_test_orig, y_pred)
rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred))
r2 = r2_score(y_test_orig, y_pred)
mape = np.mean(np.abs((y_test_orig - y_pred) / y_test_orig)) * 100

print("Линейная регрессия (Гипотеза 3: log цели)")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")
print(f"MAPE: {mape:.2f}")

Зарплата (min/max): 338096 / 3974203
Log зарплаты (min/max): 12.73 / 15.20
Линейная регрессия (Гипотеза 3: log цели)
MAE:  394710.10
RMSE: 496156.72
R²:   0.5311
MAPE: 36.70


**Вывод по гипотезе 3:**

Логарифмирование зарплаты сработало 50 на 50. С одной стороны, MAPE заметно снизился 36.70% против 39.19% в первой гипотезе. Это логично: логарифм сглаживает большие зарплаты, и модель начинает меньше ошибаться в процентах.

С другой стороны, R² (0.5311) и MAE (394 710) стали даже чуть хуже, чем в гипотезе 1 (там было R² 0.5437 и MAE 391 436). То есть в рублях мы ошибаемся чуть сильнее, но относительная ошибка стала меньше. Приём полезный, но не идеальный, выигрываем только в одной метрике.

Гипотезу 3 не берём.

In [14]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Загружаем датасет
df = pd.read_csv('Job_Market_India.csv')
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)

# 1. Удаление выбросов
q1 = df['Salary_INR'].quantile(0.01)
q99 = df['Salary_INR'].quantile(0.99)
df = df[(df['Salary_INR'] >= q1) & (df['Salary_INR'] <= q99)]
df = df.fillna(df.median(numeric_only=True))

# 2. Добавляем полиномиальные признаки для Demand_Index
# Просто возведем в квадрат вручную
df['Demand_Index_Squared'] = df['Demand_Index'] ** 2

# взаимодействие спроса и удаленки
df['Demand_Remote_Interact'] = df['Demand_Index'] * df['Remote_Option_Flag']

print("Новые признаки:")
print(df[['Demand_Index', 'Demand_Index_Squared', 'Demand_Remote_Interact']].head())

# 3. One-hot encoding
df = pd.get_dummies(df, columns=['Company_Name', 'Job_Role', 'Experience_Level', 'City'], drop_first=False)

X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']

# Train / Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Обучение
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

y_pred = linreg.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print("Линейная регрессия (Гипотеза 4: полиномиальные признаки)")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")
print(f"MAPE: {mape:.2f}")

Новые признаки:
   Demand_Index  Demand_Index_Squared  Demand_Remote_Interact
0            71                  5041                       0
1            81                  6561                       0
2            94                  8836                       0
3            94                  8836                      94
4            35                  1225                       0
Линейная регрессия (Гипотеза 4: полиномиальные признаки)
MAE:  391484.47
RMSE: 489417.00
R²:   0.5437
MAPE: 39.19


**Вывод по гипотезе 4:**

Добавление полиномиальных признаков (Demand_Index^2) и взаимодействия спроса с удалёнкой не дало никакого прироста. Метрики остались абсолютно такими же (R² = 0.5437, MAE = 391 484). Видимо, зависимость зарплаты от спроса либо достаточно линейная, либо эти признаки просто теряются на фоне остальных факторов (города, компании). Усложнять модель этими признаками нет смысла.

Гипотеза 4 отвергается.

c. и d. Формирование улучшенного бейзлайна и обучение модели

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 1. Загрузка и очистка
df = pd.read_csv('Job_Market_India.csv')
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)

# Удаление выбросов (1-99 перцентиль)
q1 = df['Salary_INR'].quantile(0.01)
q99 = df['Salary_INR'].quantile(0.99)
df = df[(df['Salary_INR'] >= q1) & (df['Salary_INR'] <= q99)]

# Заполнение пропусков
df = df.fillna(df.median(numeric_only=True))

# 2. One-hot encoding (основа улучшенного бейзлайна)
df = pd.get_dummies(df, columns=['Company_Name', 'Job_Role', 'Experience_Level', 'City'], drop_first=False)

# Разделение
X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Обучение
linreg_best = LinearRegression()
linreg_best.fit(X_train_scaled, y_train)

print("Модель с улучшенным бейзлайном обучена.")

Модель с улучшенным бейзлайном обучена.


**e. Оценка качества моделей с улучшенным бейзлайном**

In [16]:
# Предсказание
y_pred_best = linreg_best.predict(X_test_scaled)

# Метрики
mae = mean_absolute_error(y_test, y_pred_best)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_best))
r2 = r2_score(y_test, y_pred_best)
mape = np.mean(np.abs((y_test - y_pred_best) / y_test)) * 100

print("Линейная регрессия (Улучшенный бейзлайн)")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")
print(f"MAPE: {mape:.2f}")

Линейная регрессия (Улучшенный бейзлайн)
MAE:  391436.04
RMSE: 489428.13
R²:   0.5437
MAPE: 39.19


**f. Сравнение результатов с пунктом 2**

Базовый:

MAE: 593 488.92

RMSE: 791 551.53

R²: 0.0047

MAPE: 60.97%

После улучшений:

MAE: 391 436.04

RMSE: 489 428.13

R²: 0.5437

MAPE: 39.19%

Все четыре метрики существенно улучшились. Основной прирост дало именно правильное кодирование категорий: Label Encoding заставлял модель искать зависимость между зарплатой и номерами городов/компаний, чего на самом деле не существует. One-hot разбил каждую категорию на отдельные бинарные признаки, и модель начала нормально работать.

**e. Выводы**

В линейной регрессии очень важно правильно кодировать категориальные признаки. С Label Encoding модель почти не работала, R² был около нуля. Когда я перешёл на One-hot, R² сразу вырос до 0.54, и модель начала лучше объяснять данные.

Удаление выбросов тоже сильно помогло. Слишком большие и слишком маленькие зарплаты мешали модели, и после их удаления ошибки заметно снизились. MAPE уменьшился с 61% до 39%.

В итоге линейная регрессия стала работать гораздо лучше, хотя точность всё ещё средняя, но для такой модели это нормально.

# 4. Имплементация алгоритма машинного обучения

**a. Имплементация линейной регрессии**

In [1]:
import numpy as np

class LinearRegressionCustom:

    def __init__(self, lr=0.01, n_iter=1000):
        """
        lr: скорость обучения
        n_iter: количество итераций градиентного спуска
        """
        self.lr = lr
        self.n_iter = n_iter
        self.w = None
        self.b = None

    def fit(self, X, y):
        """
        Обучение модели на данных X, y.
        X: матрица признаков (numpy array)
        y: целевая переменная (numpy array)
        """
        n_samples, n_features = X.shape

        # Инициализируем веса нулями
        self.w = np.zeros(n_features)
        self.b = 0.0

        # Градиентный спуск
        for _ in range(self.n_iter):
            # Предсказания модели
            y_pred = X @ self.w + self.b

            # Градиенты по w и b
            dw = (2 / n_samples) * (X.T @ (y_pred - y))
            db = (2 / n_samples) * np.sum(y_pred - y)

            # Обновляем параметры
            self.w -= self.lr * dw
            self.b -= self.lr * db

        print("Обучение завершено")
        print("Форма вектора весов:", self.w.shape)

    def predict(self, X):
        """
        Предсказание значений для новых объектов X.
        """
        return X @ self.w + self.b

**b. и c. Обучение и оценка имплементированной модели с базовым бейзлайном**

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Загружаем датасет
df = pd.read_csv('Job_Market_India.csv')
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)

# Простой Label Encoding для категориальных признаков
cat_cols = ['Company_Name', 'Job_Role', 'Experience_Level', 'City']
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# Заполняем пропуски медианой по числовым колонкам
df = df.fillna(df.median(numeric_only=True))

# Признаки и целевая переменная
X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']

# Train / Test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

# Масштабирование признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Преобразуем в numpy-массивы
X_train_scaled = np.array(X_train_scaled)
X_test_scaled = np.array(X_test_scaled)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Обучаем
linreg_custom_base = LinearRegressionCustom(lr=0.01, n_iter=1000)
linreg_custom_base.fit(X_train_scaled, y_train)

# Предсказания и метрики
y_pred_base = linreg_custom_base.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred_base)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_base))
r2 = r2_score(y_test, y_pred_base)
mape = np.mean(np.abs((y_test - y_pred_base) / y_test)) * 100

print("Линейная регрессия с имплементацией (базовый бейзлайн)")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")
print(f"MAPE: {mape:.2f}")

Обучение завершено
Форма вектора весов: (6,)
Линейная регрессия с имплементацией (базовый бейзлайн)
MAE:  593488.91
RMSE: 791551.53
R²:   0.0047
MAPE: 60.97


**d. Сравнение результатов с пунктом 2**

**Имплементированный:**

MAE:  593488.91

RMSE: 791551.53

R²:   0.0047

MAPE: 60.97

**Sklearn:**

MAE:  593488.92

RMSE: 791551.53

R²:   0.0047

MAPE: 60.97

Результаты преедельно схожи, если не сказать одинаковы.

**e. Выводы**

Реализованный класс линейной регрессии показывает себя так же, как и стандартная библиотечная модель: на сырых данных с Label Encoding обе работают плохо. Это подтверждает, что проблема была не в алгоритме, а в подготовке данных.

**f. и g. Добавление техник из улучшенного бейзлайна и обучение имплементированной модели**

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Загружаем датасет
df = pd.read_csv('Job_Market_India.csv')
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)

# Удаляем выбросы (1-99 перцентили)
q1 = df['Salary_INR'].quantile(0.01)
q99 = df['Salary_INR'].quantile(0.99)
df = df[(df['Salary_INR'] >= q1) & (df['Salary_INR'] <= q99)]

# Заполняем пропуски
df = df.fillna(df.median(numeric_only=True))

# One-hot encoding для категориальных признаков
df = pd.get_dummies(
    df,
    columns=['Company_Name', 'Job_Role', 'Experience_Level', 'City'],
    drop_first=False
)

# Разделяем на признаки и целевую переменную
X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']

# Train / Test разделение
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

# Масштабирование признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Преобразуем в numpy массивы
X_train_scaled = np.array(X_train_scaled)
X_test_scaled = np.array(X_test_scaled)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Обучаем
# Увеличиваем количество итераций для лучшей сходимости
linreg_custom_best = LinearRegressionCustom(lr=0.01, n_iter=2000)
linreg_custom_best.fit(X_train_scaled, y_train)

print("Линейная регрессия (улучшенный бейзлайн) обучена")


Обучение завершено
Форма вектора весов: (66,)
Линейная регрессия (улучшенный бейзлайн) обучена


**h. Оценка качества моделей с улучшенным бейзлайном**

In [6]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Предсказания
y_pred_best_custom = linreg_custom_best.predict(X_test_scaled)

# Метрики
mae = mean_absolute_error(y_test, y_pred_best_custom)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_best_custom))
r2 = r2_score(y_test, y_pred_best_custom)
mape = np.mean(np.abs((y_test - y_pred_best_custom) / y_test)) * 100

print("Имплементированная линейная регрессия (улучшенный бейзлайн)")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")
print(f"MAPE: {mape:.2f}")


Имплементированная линейная регрессия (улучшенный бейзлайн)
MAE: 391436.04
RMSE: 489428.13
R²: 0.5437
MAPE: 39.19


**i. Сравнить результаты моделей в сравнении с результатами из пункта 3**

Сравнил метрики своей реализации LinearRegressionCustom на улучшенном бейзлайне с тем, что выдавал sklearn в пункте 3. Цифры получились абсолютно такие же: MAE около 391 тысячи, R² тот же самый 0.5437, да и MAPE совпал (39.19%).​

Получается, что моя модель обучилась так же хорошо, как и библиотечная. Разницы в предсказаниях нет вообще, значит, алгоритм работает корректно и сходится к тому же оптимальному решению, что и в sklearn.

**j. Выводы**


Собственная линейная регрессия полностью справилась с задачей. На улучшенных данных (без выбросов и с One-hot encoding) она показала ровно такое же качество, как и готовая модель из библиотеки.​